In [5]:
%pwd

'd:\\Comapy\\Projects\\CureNow\\research'

In [6]:
import os


os.chdir("../")

In [7]:
%pwd

'd:\\Comapy\\Projects\\CureNow'

# Load Environment Variables

In [8]:
from dotenv import load_dotenv
import os

load_dotenv()

GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")

In [9]:
print("HI")

HI


# Import Necessary Libraries

In [10]:
import google.generativeai as genai
from langchain.document_loaders import PyPDFLoader,DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone,ServerlessSpec

# Load PDF Data

In [11]:
#Extract the data from the pdf

def load_pdf_file(data):
    loader = DirectoryLoader(data,
                             glob="*.pdf",
                             loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents
extracted_data = load_pdf_file(data="Data/")

# Split Text into Chunks

In [12]:
# pip list - to see the downloaded version of the library
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000,chunk_overlap = 100)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

text_chunks = text_split(extracted_data)
print(f"Length of text chunks = {len(text_chunks)}")

Length of text chunks = 4471


# Generate Embedding

In [26]:
# def download_hugging_face_embeddings():
#     embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
#     return embeddings

# embeddings = download_hugging_face_embeddings()
# query_result = embeddings.embed_query("Hello World!")
# print("Length",len(query_result))

from transformers import AutoTokenizer, AutoModel
import torch
# Load 1024-dimensional embedding model
def get_1024d_embeddings(text):
    model_name = "intfloat/e5-large"  # 1024-d embedding model
    model = AutoModel.from_pretrained(model_name)
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    inputs = tokenizer(text, return_tensors="pt")
    with torch.no_grad():
        text_features = model(**inputs).last_hidden_state.mean(dim=1)  # 1024-d vector

    return text_features.squeeze().numpy()

# Example query
query_result = get_1024d_embeddings("Hello World!")
print("Length:", len(query_result))  # Should be 1024

# Use Hugging Face 1024-d embeddings for Pinecone
embeddings = HuggingFaceEmbeddings(model_name="intfloat/e5-large")

c:\Users\ASUS\anaconda3\envs\curehealth\lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ASUS\.cache\huggingface\hub\models--intfloat--e5-large. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Length: 1024


# Initialize Pinecone

In [27]:
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY')
# OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
GEMINI_API_KEY = os.environ.get("GEMINI_API_KEY")

In [21]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key=PINECONE_API_KEY)
# index_name = "curenow1"
index_name = "curenow-chatbot"
pc.create_index(
    name=index_name,
    dimension=3072, # Replace with your model dimensions
    metric="cosine", # Replace with your model metric
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)

PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2024-04', 'X-Cloud-Trace-Context': '965a3f0ecfc18053cbf8d718a710caaa', 'Date': 'Mon, 10 Mar 2025 04:46:06 GMT', 'Server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [28]:
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
# os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
os.environ["GEMINI_API_KEY"] = GEMINI_API_KEY

In [29]:
#Initializing the Pinecone
Pinecone(api_key=PINECONE_API_KEY,environment = "us-east-1"
              )

# Store Data in Pinecone

In [ ]:
# from langchain_pinecone import PineconeVectorStore

# docresearch = PineconeVectorStore.from_documents(
#     documents=text_chunks,
#     index_name=index_name,
#     embedding=embeddings,
# )


# Load Exiting Index

In [31]:
#Load Existing index

from langchain_pinecone import PineconeVectorStore

#embed each chunk and upsert the embeddings into your Pinecone index.

docresearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [32]:
retriever = docresearch.as_retriever(search_type = 'similarity',search_kwargs = {"k" : 3})


In [33]:
retrieved_docs = retriever.invoke("What is Acne?")


In [34]:
retrieved_docs

[Document(id='e315b231-57ba-4634-b05a-a148d4f202ed', metadata={'page': 37.0, 'page_label': '38', 'source': 'Data\\Medical_book.pdf'}, page_content='Resources\nBOOKS\nA Manual of Laboratory and Diagnostic Tests.5th ed. Ed.\nFrancis Fishback. Philadelphia: Lippincott, 1996.\nGarza, Diana, and Kathleen Becan-McBride. Phlebotomy\nHandbook. 4th ed. Stamford: Appleton & Lange, 1996.\nPERIODICALS\nMoul, Judd W., et al. “The Contemporary Value of Pretreat-\nment Prostatic Acid Phosphatase to Predict Pathological\nStage and Recurrence in Radical Prostatectomy Cases.”\nJournal of Urology(Mar. 1998): 935-940.\nNancy J. Nordenson\nAcid reflux see Heartburn\nAcidosis see Respiratory acidosis; Renal\ntubular acidosis; Metabolic acidosis\nAcne\nDefinition\nAcne is a common skin disease characterized by\npimples on the face, chest, and back. It occurs when the\npores of the skin become clogged with oil, dead skin\ncells, and bacteria.\nDescription\nAcne vulgaris, the medical term for common acne, is\n

In [ ]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "cure-450112-a6ef848507da.json"


In [ ]:
# from langchain_openai import OpenAI

# llm = OpenAI(temperature=0.4,max_tokens=400)
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-lite-preview-02-05",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

# https://python.langchain.com/docs/integrations/chat/google_generative_ai/
# https://aistudio.google.com/prompts/new_chat?_gl=1*1g9u9qo*_ga*MTE4NjAyOTczOS4xNzM4ODIwNjQ1*_ga_P1DBVKWT6V*MTczODgzMDg1MC4yLjAuMTczODgzMDg1MC42MC4wLjEzMjM2Njk1MDI.


In [ ]:
# # Create the model
# generation_config = {
#   "temperature": 1,
#   "top_p": 0.95,
#   "top_k": 64,
#   "max_output_tokens": 8192,
#   "response_mime_type": "text/plain",
# }

# model = genai.GenerativeModel(
#   model_name="gemini-2.0-flash-lite-preview-02-05",
#   generation_config=generation_config,
# )

# Use Gemini API for Response Generation

In [ ]:
# genai.configure(api_key=GEMINI_API_KEY)
# model = genai.GenerativeModel("gemini-pro")

In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

In [30]:
# Set up Pinecone vector store (adjust batch_size if necessary)
pinecone_index_name = "curenow"

# If you're still using Pinecone for vector storage, update the index and embedding
pinecone_vector_store = PineconeVectorStore.from_documents(
    documents=text_chunks,  # Replace 'text_chunks' with actual document content
    embedding=embeddings,
    batch_size=20,  # Adjust batch size to prevent timeouts
    index_name=pinecone_index_name
)

In [ ]:
# llm = GoogleGenerativeAIEmbeddings(model="gemini-pro", google_api_key=GEMINI_API_KEY)


In [31]:

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following piences of retrieved context to answer "
    "the question.If you don't know the answer ,say that you "
    "don't know.Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system",system_prompt),
        ("human","{input}"),
    ]
)


In [32]:
# llm = OpenAI(temperature=0.4, max_tokens=400)


In [33]:
question_answer_chain = create_stuff_documents_chain(llm,prompt)
rag_chain = create_retrieval_chain(retriever,question_answer_chain)
# rag_chain = create_retrieval_chain(pinecone_vector_store,question_answer_chain)

In [34]:
# Retrieve and Answer Query
response = rag_chain.invoke({"input": " what are the chronical diseases in a very simple terms understandble by non doctors "})
print(response["answer"])

A chronic condition is a long-lasting health issue. These conditions often develop slowly and involve gradual changes in the body. Examples include glaucoma and depression.


In [35]:
retriever = docresearch.as_retriever(search_type='similarity', search_kwargs={'k': 5})


In [36]:
system_prompt = (
    "You are an AI assistant. Use the retrieved documents to answer the question. "
    "If the answer is not in the documents, say 'The provided documents do not contain enough information.' "
    "Provide a concise and clear response."
    "\n\n"
    "{context}"
)
for doc in retrieved_docs:
    print(doc.page_content)  # Ensure the content has useful information


Isotretinoin (Accutane) is prescribed only for very
severe, disfiguring acne.
Acne is a skin condition that occurs when pores or
hair follicles become blocked. This allows a waxy
material, sebum, to collect inside the pores or follicles.
Normally, sebum flows out onto the skin and hair to
form a protective coating, but when it cannot get out,
small swellings develop on the skin surface. Bacteria
and dead skin cells can also collect that can cause
inflammation. Swellings that are small and not
inflamed are whiteheads or blackheads. When they
become inflamed, they turn into pimples. Pimples that
fill with pus are called pustules.
Acne cannot be cured, but acne drugs can help clear
the skin. Benzoyl peroxide and tretinoin work by mildly
irritating the skin. This encourages skin cells to slough
off, which helps open blocked pores. Benzoyl peroxide
also kills bacteria, which helps prevent whiteheads and
blackheads from turning into pimples. Isotretinoin
Isotretinoin (Accutane) is prescribed